In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_pickle("./dataframe/df_only_risky_events.pkl")
data.reset_index(inplace=True)
data.drop(['index'], inplace=True, axis=1)
print(data.shape)
data.head()

(272, 86)


,__time_to_tca,MISS_DISTANCE,RELATIVE_SPEED,RELATIVE_POSITION_R,RELATIVE_POSITION_T,RELATIVE_POSITION_N,RELATIVE_VELOCITY_R,RELATIVE_VELOCITY_T,RELATIVE_VELOCITY_N,COLLISSION_PROBABILITY,...,PC_mavg_1,PC_trend_1,PC_trend_3,PC_gradient_1,PC_gradient_3,MD_mavg_1,MD_trend_1,MD_trend_3,MD_gradient_1,MD_gradient_3
0,0.832257,2244.0,11172.0,87.9,-1474.9,1689.9,-201.4,-8424.3,-7336.0,-3.418392,...,-3.475175,0.132439,0.590914,0.354472,0.579354,2181.333333,47.0,17.0,125.795356,16.667421
1,0.580855,966.0,212.0,191.0,-944.8,-68.9,33.8,-8.4,209.3,-3.711974,...,-3.981494,0.310256,0.575376,0.747481,0.536846,1131.000000,-201.0,-326.0,-484.256505,-304.169574
2,0.189825,1102.0,212.0,193.3,-1083.1,-75.0,33.9,-8.4,209.3,-3.913996,...,-3.882734,-0.202022,0.296282,-0.516640,0.270366,1078.333333,136.0,-158.0,347.799165,-144.179478
3,0.994637,456.0,11707.0,-22.1,288.8,353.3,-10.2,-9077.7,7392.6,-2.055073,...,-2.841298,1.280567,1.312874,3.634346,1.445698,709.666667,-577.0,124.0,-1637.570088,136.545083
4,0.759012,629.0,11707.0,-18.7,395.3,489.9,-10.3,-9077.7,7392.6,-2.649558,...,-2.680091,-0.594485,0.483623,-2.523011,0.486607,706.000000,173.0,-11.0,734.217036,-11.067877


In [3]:
train, test = train_test_split(data, test_size=0.25, random_state=42)

In [4]:
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))

Train dataframe dimension 204 x 86
Test dataframe dimension 68 x 86


In [5]:
Y_train = train["COLLISSION_PROBABILITY"]
X_train= train.drop(["COLLISSION_PROBABILITY"], axis=1)
Y_test = test["COLLISSION_PROBABILITY"]
X_test= test.drop(["COLLISSION_PROBABILITY"], axis=1)


In [6]:
X = X_train
y = Y_train

In [7]:
#print(list(X.columns))

In [8]:
# from sklearn.model_selection import RepeatedKFold
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import AdaBoostRegressor
# import datetime as dt
# import pickle

# # define the model with default hyperparameters
# model = AdaBoostRegressor()
# # define the grid of values to search
# grid = dict()
# #grid['n_estimators'] = [10, 50, 100, 500,1000,1500,2000,3000]
# grid['n_estimators'] = [10, 50]
# grid['learning_rate'] = [0.0001, 0.001]
# # define the evaluation procedure
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# # define the grid search procedure
# grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='r2')

# # execute the grid search
# grid_result = grid_search.fit(X, y)

# # summarize the best score and configuration
# now=dt.datetime.now()
# filename1="./opt_parameters_ada_boost/{}_gs_opt_param.pkl".format(
#         now.strftime("%Y%m%d_%H%M%S"))
# filename2="./opt_parameters_ada_boost/{}_gs_full_run.txt".format(
#         now.strftime("%Y%m%d_%H%M%S"))
# with open(filename1, "wb") as optimal_parameters_logger:
#     output_dict = grid_result.best_params_
#     pickle.dump(output_dict, optimal_parameters_logger)
#     optimal_parameters_logger.close()
# with open(filename2, "a") as results_logger:
#     output_1 = "Best: {} using {} \n".format(grid_result.best_score_, grid_result.best_params_)
#     print(output_1)
#     results_logger.write(output_1)
#     # summarize all scores that were evaluated
#     means = grid_result.cv_results_['mean_test_score']
#     stds = grid_result.cv_results_['std_test_score']
#     params = grid_result.cv_results_['params']
#     for mean, stdev, param in zip(means, stds, params):
#         output_2 = "{} ({}) with: {} \n".format(mean, stdev, param)
#         #print("%f (%f) with: %r" % (mean, stdev, param))
#         print(output_2)
#         results_logger.write(output_2)
#     results_logger.close()

In [9]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
import datetime as dt
import pickle


def grid_search_optimization(
    X,
    y,
    n_estimators_list=[10, 50],
    learning_rate_list=[0.0001, 0.001],
    loss_functions_list = ["linear"],
    scoring_metric="r2",
    n_splits_for_cv = 5,
    n_repeats_for_cv = 3
):
    """Computes the optimal values for the LGBM model

    Parameters
    ----------
    X : dataframe
        Train dataset
    y : target dataframe
        Values to be predicted
    n_estimators_list: list
        List of n_estimators values for grid search
    learning_rate_list: list
        List of learning_rate values for grid search
    scoring_metric: 
        Scoring metrics from sci-kit learn default r2
    n_splits_for_cv:
        Number of splits for K Fold cross validation
    n_repeats_for_cv:
        Number of repetition for cross validation

    Returns
    -------
    dictionary
        Optimized values
    """
    # define the model with default hyperparameters
    model = AdaBoostRegressor()
    # define the grid of values to search
    grid = dict()
    #grid['n_estimators'] = [10, 50, 100, 500,1000,1500,2000,3000]
    grid['n_estimators'] = n_estimators_list
    grid['learning_rate'] = learning_rate_list
    grid['loss'] = loss_functions_list
    # define the evaluation procedure
    cv = RepeatedKFold(n_splits=n_splits_for_cv, n_repeats=n_repeats_for_cv, random_state=1)
    # define the grid search procedure
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring_metric)

    # execute the grid search
    grid_result = grid_search.fit(X, y)

    # summarize the best score and configuration
    now=dt.datetime.now()
    filename1="./opt_parameters_ada_boost/{}_{}_gs_opt_param.pkl".format(
            now.strftime("%Y%m%d_%H%M%S"),scoring_metric)
    filename2="./opt_parameters_ada_boost/{}_gs_full_run.txt".format(
            now.strftime("%Y%m%d_%H%M%S"),scoring_metric)
    with open(filename1, "wb") as optimal_parameters_logger:
        output_dict = grid_result.best_params_
        output_dict["scoring"] = scoring_metric
        output_dict["best_score"] = grid_result.best_score_
        output_dict["n_splits"] = n_splits_for_cv
        output_dict["n_repeats"] = n_repeats_for_cv
        pickle.dump(output_dict, optimal_parameters_logger)
        optimal_parameters_logger.close()
    with open(filename2, "a") as results_logger:
        output_1 = "Best: {} using {} \n".format(grid_result.best_score_, grid_result.best_params_)
        print(output_1)
        results_logger.write(output_1)
        # summarize all scores that were evaluated
        means = grid_result.cv_results_['mean_test_score']
        stds = grid_result.cv_results_['std_test_score']
        params = grid_result.cv_results_['params']
        for mean, stdev, param in zip(means, stds, params):
            output_2 = "{} ({}) with: {} \n".format(mean, stdev, param)
            print(output_2)
            results_logger.write(output_2)
        results_logger.close()
    return output_dict

In [10]:
n_estimators_v=list(range(50,4025,25))
learning_rate_v=[0.001,0.002,0.0025,0.005,0.0075,0.01,0.0125,0.015,0.0175,0.02,0.025,0.03,0.04,0.05]
loss_function_v = ["linear","exponential"]
regression_metrics=["r2","neg_mean_absolute_error","neg_mean_squared_error","neg_root_mean_squared_error"]

In [11]:
for i in regression_metrics:
    grid_search_optimization(X,y,n_estimators_v,learning_rate_v,scoring_metric=i,loss_functions_list=loss_function_v)

KeyboardInterrupt: 